In [ ]:
import re
import pennylane as qml
from pennylane import FermiC, FermiA
from pennylane import numpy as np
import itertools

# Manual orbital reordering to go from block labeling of indicies to Pennylane ordering
reordering_map = {
    0: 0, 1: 1, 2: 2, 3: 6, 4: 7, 5: 8,
    6: 3, 7: 4, 8: 5, 9: 9, 10: 10, 11: 11
}

# Parse fermionic Hamiltonian terms into (coefficient, operator list)
def parse_term(term_str):
    match = re.match(r"\(([^)]+)\)(.*)", term_str.strip())
    if not match:
        raise ValueError(f"Invalid format: {term_str}")
    
    coeff_str, ops_str = match.groups()
    coeff = float(coeff_str.split(',')[0])
    
    ops = []
    if ops_str.strip():  
        for op in ops_str.strip().split():
            if op.endswith("^"):
                idx = int(op[:-1])
                dagger = True
            else:
                idx = int(op)
                dagger = False
            remapped_idx = reordering_map[idx]
            ops.append((remapped_idx, dagger))
    
    return coeff, ops

# Load Hamiltonian from external text file
def load_hamiltonian(filename):
    with open(filename, 'r') as f:
        content = f.read()
    terms = [t.strip() for t in content.split('+') if t.strip()]
    return [parse_term(t) for t in terms]

# Convert into Pennylane fermionic operators
def build_fermionic_operator(parsed_terms):
    total_op = 0
    for coeff, ops in parsed_terms:
        op = None
        for idx, dagger in ops:
            term = FermiC(idx) if dagger else FermiA(idx)
            op = term if op is None else op * term
        if op is not None:
            total_op += float(coeff) * op
        else:
            total_op += float(coeff)  # constant/identity term
    return total_op

parsed_terms = load_hamiltonian("Benzene_VDZ_ferm.txt")

fermionic_op = build_fermionic_operator(parsed_terms)
#print(fermionic_op)

# Jordan-Wigner to convert to qubit Hamiltonian
jw_op = qml.fermi.jordan_wigner(fermionic_op)

# Reconstruct the Hamiltonian with only real parts of the coefficients
H = sum(c.real * t for c, t in zip(jw_op.coeffs, jw_op.ops))

qubits = 12
electrons = 6
wires = list(range(qubits))

# Construct Hartree-Fock state
hf_state = qml.qchem.hf_state(electrons, qubits)

# Generate generalized singles and doubles excitations
combinations = list(itertools.combinations(range(12), 2))
singles = [list(combo) for combo in combinations]
combinations = list(itertools.combinations(range(12), 4))
doubles = [list(combo) for combo in combinations]

dev = qml.device("lightning.qubit", wires=qubits)

# Construct circuit with 2 repetition UCCGSD ansatz 
def circuit_1(params, *, excitations):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i+len(doubles)], wires=excitation)
        else:
            qml.SingleExcitation(params[i+len(singles)], wires=excitation)
    return qml.expval(H)

cost_fn = qml.QNode(circuit_1, dev)

# Initialize parameters
params = np.zeros(2*(len(doubles)+len(singles)))

# Choose optimizer
optimizer = qml.AdagradOptimizer(stepsize=1.5, eps=1e-08)

# Carry out optimization

max_steps = 20000  

for m in range(max_steps):
    params, energy = optimizer.step_and_cost(cost_fn, params, excitations=singles+doubles)
    
    if m % 2 == 0:
        print(f"Step = {m},  E = {float(energy):.8f} Ha")



Step = 0,  E = -231.55004881 Ha
Step = 2,  E = -230.38627863 Ha
Step = 4,  E = -230.46092129 Ha
Step = 6,  E = -230.71069694 Ha
Step = 8,  E = -230.76954409 Ha
Step = 10,  E = -230.71554843 Ha
Step = 12,  E = -230.55042127 Ha
Step = 14,  E = -230.74336487 Ha
Step = 16,  E = -230.75992323 Ha
Step = 18,  E = -231.03810313 Ha
Step = 20,  E = -230.87303418 Ha
Step = 22,  E = -231.04431522 Ha
Step = 24,  E = -230.95281346 Ha
Step = 26,  E = -230.99125032 Ha
Step = 28,  E = -231.08862201 Ha
Step = 30,  E = -231.13994534 Ha
Step = 32,  E = -231.27328940 Ha
Step = 34,  E = -231.16454059 Ha
Step = 36,  E = -231.16017578 Ha
Step = 38,  E = -231.22857809 Ha
Step = 40,  E = -231.02503999 Ha
Step = 42,  E = -231.19006884 Ha
Step = 44,  E = -231.29203387 Ha
Step = 46,  E = -231.12514517 Ha
Step = 48,  E = -231.25100521 Ha
Step = 50,  E = -231.23994135 Ha
Step = 52,  E = -231.11068156 Ha
Step = 54,  E = -231.19227692 Ha
Step = 56,  E = -231.21111850 Ha
Step = 58,  E = -231.17419090 Ha
Step = 60,  E =

KeyboardInterrupt: 